In [1]:
from scipy.spatial.transform import Rotation as R # Magnitude (and '*_matrix', use '*_dcm' instead) require a newer version of SciPy than SPIN's
import numpy as np
import torch

In [2]:
# Let's test this by loading up a real example from 3dpw

import pickle as pkl

img_path = 'examples/image_00502_crop.jpg'
#img_path = 'examples/image_00980.jpg'

pickle_path = 'data/3dpw/sequenceFiles/validation/courtyard_basketball_01.pkl'
#pickle_path = 'data/3dpw/sequenceFiles/validation/outdoors_parcours_01.pkl'

frame = 502

In [3]:
# using IPython's Magic %run let's us cleanly run the Demo script and get the variables defined
# therein into this notebook. use $ to insert variables from the notebook into any %magic command
%run demo.py --checkpoint=data/model_checkpoint.pt --img=$img_path

In [4]:
# open the sequence file, fetch the body_pose of the corresponding frame from it, remove global orientation
# and reshape it from (69,1) to (23,3) to input into a rotation object

seq = pkl.load(open(pickle_path,'rb'),encoding='latin-1')
gt_pose_axis_angle = seq['poses'][0][frame][3:]
gt_pose = R.from_rotvec(np.reshape(gt_pose_axis_angle, (23,-1)))

In [5]:
# create a rotation object from the predicted pose output of demo.py

pred_pose = R.from_dcm(pred_output.body_pose.squeeze().cpu())

In [6]:
# show the difference between the predicted and the ground truth pose

#e = R.as_euler(gt_pose, 'xyz', degrees=True)
#e

In [7]:
# Now, let's check through examples that this behavior makes sense

#Left hip [flexion(front kick), external rotation, abduction]
#Right hip [extension(back kick), internal rotation, adduction]
#Spine [flexion (crunch),  rotate to the left, touch right toe]
#Left knee [flexion, external rotation, abduction]
#Right knee [extension, internal rotation, adduction]
#Torso [flexion (crunch), rotate to the left, touch right shin]
#Left ankle [flexion, external rotation, abduction]
#Right ankle [flexion, internal rotation, adduction]
#Chest [flexion (crunch), rotate to the left, touch right knee]
#Left toes [flexion, fibular deviation, pronation]
#Right toes [flexion, tibial deviation, supination]
#Neck [flexion, rotate to the left, touch right shoulder]
#Left scapula [internal rotation, rotate backwards, lift arm]
#Right scapula [internal rotation, rotate forward, lower arm]
#Skull [flexion, look to the left, touch right shoulder]
#Left shoulder [internal rotation, rotate backwards, lift arm]
#Right shoulder [internal rotation, rotate forward, lower arm]
#Left elbow [internal rotation, hyperextension, abduction (unnatural)]
#Right elbow [internal rotation, flexion, adduction (unnatural)]
#Left wrist [interal rotation, ulnar deviation, extension]
#Right wrist [internal rotation, radial deviation, flexion]
#Left knuckles [internal rotation, ulnar deviation (unnatural), hyperextension]
#Right knuckles [internal rotation, radial deviation, flexion]

In [3]:
d = {
     'Left hip':{'Name': 'Left hip', 'x': 'Flexion', 'y': 'External rotation', 'z': 'Abduction'},
     'Right hip':{'Name': 'Right hip', 'x': 'Extension', 'y': 'Internal rotation', 'z': 'Adduction'},
     'Spine':{'Name': 'Spine', 'x': 'Flexion', 'y': 'Rotate to the left', 'z': 'Touch right ankle'},
     'Left knee':{'Name': 'Left knee', 'x': 'Flexion', 'y': 'External rotation', 'z': 'Abduction'},
     'Right knee':{'Name': 'Right knee', 'x': 'Extension', 'y': 'External rotation', 'z': 'Adduction'},
     'Torso':{'Name': 'Torso', 'x': 'Flexion', 'y': 'Rotate to the left', 'z': 'Touch right shin'},
     'Left ankle':{'Name': 'Left ankle', 'x': 'Flexion', 'y': 'External rotation', 'z': 'Abduction'},
     'Right ankle':{'Name': 'Right ankle', 'x': 'Flexion', 'y': 'Internal rotation', 'z': 'Adduction'},
     'Chest':{'Name': 'Chest', 'x': 'Flexion', 'y': 'Rotate to the left', 'z': 'Touch right knee'},
     'Left toes':{'Name': 'Left toes', 'x': 'Flexion', 'y': 'Fibular deviation', 'z': 'Pronation'},
     'Right toes':{'Name': 'Right toes', 'x': 'Flexion', 'y': 'Tibial deviation', 'z': 'Supination'},
     'Neck':{'Name': 'Neck', 'x': 'Flexion', 'y': 'Rotate to the left', 'z': 'Touch right shoulder'},
     'Left scapula':{'Name': 'Left scapula', 'x': 'Internal rotation', 'y': 'Rotate backwards', 'z': 'Clockwise rotation'},
     'Right scapula':{'Name': 'Right scapula', 'x': 'Internal rotation', 'y': 'Rotate forward', 'z': 'Clockwise rotation'},
     'Skull':{'Name': 'Skull', 'x': 'Flexion', 'y': 'Look to the left', 'z': 'Touch right shoulder'},
     'Left shoulder':{'Name': 'Left shoulder', 'x': 'Internal rotation', 'y': 'Rotate backwards', 'z': 'Raise arm'},
     'Right shoulder':{'Name': 'Right shoulder', 'x': 'Internal rotation', 'y': 'Rotate forward', 'z': 'Lower arm'},
     'Left elbow':{'Name': 'Left elbow', 'x': 'Internal rotation', 'y': 'Extension', 'z': 'Abduction'},
     'Right elbow':{'Name': 'Right elbow', 'x': 'Internal rotation', 'y': 'Flexion', 'z': 'Adduction'},
     'Left wrist':{'Name': 'Left wrist', 'x': 'Internal rotation', 'y': 'Ulnar deviation', 'z': 'Extension'},
     'Right wrist':{'Name': 'Right wrist', 'x': 'Internal rotation', 'y': 'Radial deviation', 'z': 'Flexion'},
     'Left knuckles':{'Name': 'Left knuckles', 'x': 'Internal rotation', 'y': 'Ulnar deviation', 'z': 'Extension'},
     'Right knuckles':{'Name': 'Right knuckles', 'x': 'Internal rotation', 'y': 'Radial deviation', 'z': 'Flexion'},
    }

In [9]:
mpjae = torch.load('mpjae_per_part.pt')
mpjae_mean = mpjae.mean(dim=0)
mpjae_mean_no_root = mpjae_mean[1:]
mpjae_mean_no_root.shape

torch.Size([23, 3])

In [11]:
e = mpjae_mean_no_root

i = 0
for key in d:    
    print('{:16}'.format(d[key]['Name']),
          '{:>20}'.format(d[key]['x']),'{:> 7.2f}'.format(e[i][0]),
          '{:>20}'.format(d[key]['y']),'{:> 7.2f}'.format(e[i][1]),
          '{:>20}'.format(d[key]['z']),'{:> 7.2f}'.format(e[i][2]))
    i+=1

Left hip                      Flexion   10.05    External rotation    5.62            Abduction    4.08
Right hip                   Extension    9.84    Internal rotation    4.61            Adduction    4.88
Spine                         Flexion    9.16   Rotate to the left    2.08    Touch right ankle    2.28
Left knee                     Flexion   16.95    External rotation    5.13            Abduction    6.10
Right knee                  Extension   16.49    External rotation    4.13            Adduction    4.92
Torso                         Flexion    4.42   Rotate to the left    2.16     Touch right shin    2.04
Left ankle                    Flexion    4.77    External rotation    8.33            Abduction    3.90
Right ankle                   Flexion    5.54    Internal rotation   11.17            Adduction    6.72
Chest                         Flexion    3.01   Rotate to the left    1.69     Touch right knee    1.01
Left toes                     Flexion   11.98    Fibular deviati

In [10]:
e = np.degrees(R.as_rotvec(pred_pose))

i = 0
for key in d:    
    print('{:16}'.format(d[key]['Name']),
          '{:>20}'.format(d[key]['x']),'{:> 7.2f}'.format(e[i][0]),
          '{:>20}'.format(d[key]['y']),'{:> 7.2f}'.format(e[i][1]),
          '{:>20}'.format(d[key]['z']),'{:> 7.2f}'.format(e[i][2]))
    i+=1

Left hip                      Flexion  -14.28    External rotation   -4.00            Abduction    4.80
Right hip                   Extension    6.29    Internal rotation   -4.64            Adduction   -7.11
Spine                         Flexion    5.57   Rotate to the left    3.83    Touch right ankle    1.37
Left knee                     Flexion   33.36    External rotation   -7.89            Abduction   -5.22
Right knee                  Extension   40.79    External rotation   10.29            Adduction    1.49
Torso                         Flexion   -2.53   Rotate to the left   -1.29     Touch right shin    0.95
Left ankle                    Flexion   -1.98    External rotation    8.60            Abduction   -7.68
Right ankle                   Flexion   -9.52    Internal rotation  -12.81            Adduction    9.81
Chest                         Flexion   -1.05   Rotate to the left    1.87     Touch right knee    0.91
Left toes                     Flexion  -10.76    Fibular deviati

In [12]:
r = R.as_rotvec(gt_pose)
q = R.as_rotvec(pred_pose)

e = np.degrees(r-q)

i = 0
for key in d:    
    print('{:16}'.format(d[key]['Name']),
          '{:>20}'.format(d[key]['x']),'{:> 7.2f}'.format(e[i][0]),
          '{:>20}'.format(d[key]['y']),'{:> 7.2f}'.format(e[i][1]),
          '{:>20}'.format(d[key]['z']),'{:> 7.2f}'.format(e[i][2]))
    i+=1

Left hip                      Flexion   17.35    External rotation    6.63            Abduction    5.19
Right hip                   Extension   12.80    Internal rotation   18.63            Adduction   -3.77
Spine                         Flexion   20.17   Rotate to the left   -4.03    Touch right ankle    4.12
Left knee                     Flexion   -5.90    External rotation    4.73            Abduction    3.65
Right knee                  Extension   -7.21    External rotation   -3.10            Adduction    0.34
Torso                         Flexion    2.25   Rotate to the left    6.70     Touch right shin   -6.27
Left ankle                    Flexion   -2.89    External rotation   -6.93            Abduction    1.65
Right ankle                   Flexion   17.40    Internal rotation   20.31            Adduction  -14.53
Chest                         Flexion   -1.87   Rotate to the left    1.37     Touch right knee   -3.94
Left toes                     Flexion   10.20    Fibular deviati

In [20]:
abs(e).mean()

12.577570935087879

In [36]:
# Let's say we now want to make our own SMPL pose with an elbow flexed at 90 degrees:

gt_pose_axis_angle = np.zeros(69)
gt_pose_axis_angle[58-3] = np.pi/2
gt_pose = R.from_rotvec(np.reshape(gt_pose_axis_angle, (23,-1)))

In [37]:
e = np.degrees(gt_pose.as_rotvec())

i = 0
for key in d:    
    print('{:16}'.format(d[key]['Name']),
          '{:>20}'.format(d[key]['x']),'{:> 7.2f}'.format(e[i][0]),
          '{:>20}'.format(d[key]['y']),'{:> 7.2f}'.format(e[i][1]),
          '{:>20}'.format(d[key]['z']),'{:> 7.2f}'.format(e[i][2]))
    i+=1

Left hip                      Flexion    0.00    External rotation    0.00            Abduction    0.00
Right hip                   Extension    0.00    Internal rotation    0.00            Adduction    0.00
Spine                         Flexion    0.00   Rotate to the left    0.00    Touch right ankle    0.00
Left knee                     Flexion    0.00    External rotation    0.00            Abduction    0.00
Right knee                  Extension    0.00    External rotation    0.00            Adduction    0.00
Torso                         Flexion    0.00   Rotate to the left    0.00     Touch right shin    0.00
Left ankle                    Flexion    0.00    External rotation    0.00            Abduction    0.00
Right ankle                   Flexion    0.00    Internal rotation    0.00            Adduction    0.00
Chest                         Flexion    0.00   Rotate to the left    0.00     Touch right knee    0.00
Left toes                     Flexion    0.00    Fibular deviati